In [ ]:
###DataCollection

In [1]:
import zipfile
import os

# Path to the uploaded ZIP file
zip_file_path = 'archive.zip'

# Folder where you want to extract the ZIP file
extract_folder = 'extract_files'

# Create the folder if it doesn't exist
os.makedirs(extract_folder, exist_ok=True)

# Extract the contents of the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"Files extracted to: {extract_folder}")


Files extracted to: extract_files


In [ ]:
###Creating training and testing dataset

In [3]:
trainPath = r"extract_files/Alzheimer_s Dataset/train"
testPath = r"extract_files/Alzheimer_s Dataset/test"

In [ ]:
###Image Preprocessing

In [ ]:
###Importing libraries

In [5]:
!pip install tensorflow
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
###Configure of Images & Preprocessing

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
IMG_SIZE = 180
IMAGE_SIZE = [180, 180]
DIM = (IMG_SIZE, IMG_SIZE)
ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = True
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"
WORK_DIR = "extract_files/Alzheimer_s Dataset/train"
work_dr = IDG(rescale = 1./225, brightness_range = BRIGHT_RANGE, zoom_range = ZOOM,  fill_mode = FILL_MODE, data_format = DATA_FORMAT, horizontal_flip = HORZ_FLIP)
train_data_gen = work_dr.flow_from_directory(directory = WORK_DIR, target_size = DIM, batch_size = 6500, shuffle = False)

Found 5121 images belonging to 4 classes.


In [ ]:
###Handling Imbalance Data

In [11]:
train_data, train_labels = next(train_data_gen)
print(train_data.shape, train_labels.shape)

(5121, 180, 180, 3) (5121, 4)


In [13]:
#After oversampling
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
train_data, train_labels = sm.fit_resample(train_data.reshape(-1, IMG_SIZE * IMG_SIZE * 3), train_labels)
train_data = train_data.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
print(train_data.shape, train_labels.shape)

(10240, 180, 180, 3) (10240, 4)


In [ ]:
###Splitting into train test split

In [17]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)

In [ ]:
###Model Building & Testing

In [ ]:
###Pre-trained CNN model as a Feature Extractor

In [19]:
IMAGE_SIZE = [180, 180]
xcep_model = Xception(input_shape=IMAGE_SIZE+[3], weights= 'imagenet',include_top=False)
for layer in xcep_model.layers:
     layer.trainable=False

In [ ]:
###Create Sequential Layers

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    SeparableConv2D, BatchNormalization, GlobalAveragePooling2D, 
    Dropout, Flatten, Dense
)
custom_inception_model = Sequential([
    xcep_model,
    Dropout(0.5),
    GlobalAveragePooling2D(),
    Flatten(),
    BatchNormalization(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(4, activation='softmax')
], name="inception_cnn_model")


In [ ]:
###Configure the Learning Process

In [29]:
custom_inception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
###Train the model

In [31]:
history = custom_inception_model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=30)

Epoch 1/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.3037 - loss: 1.7156 - val_accuracy: 0.5682 - val_loss: 1.1073
Epoch 2/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.4725 - loss: 1.2089 - val_accuracy: 0.6435 - val_loss: 0.8306
Epoch 3/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.5581 - loss: 0.9786 - val_accuracy: 0.6520 - val_loss: 0.7255
Epoch 4/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 224s 2s/step - accuracy: 0.6274 - loss: 0.8366 - val_accuracy: 0.6787 - val_loss: 0.6818
Epoch 5/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - accuracy: 0.6401 - loss: 0.7899 - val_accuracy: 0.6835 - val_loss: 0.6505
Epoch 6/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.6715 - loss: 0.7307 - val_accuracy: 0.6921 - val_loss: 0.6275
Epoch 7/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 224s 2s/step - accuracy: 0.6784 - loss: 0.7119 - val_accuracy: 0.7083 - val_loss: 0.6237
Epoch 8/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 187s 1s/step - accuracy: 0.6856 - loss: 0.6975 - val_accu

In [ ]:
###Save The Model

In [33]:
custom_inception_model.save('adp.h5')

In [ ]:
###Test The Model

In [57]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('adp.h5')
img = image.load_img(r"extract_files/Alzheimer_s Dataset/test/ModerateDemented/32.jpg", target_size=(180, 180))
x = image.img_to_array(img)
x = x / 255.0 
x = np.expand_dims(x, axis=0)
preds = model.predict(x)
pred = np.argmax(preds, axis=1)
index = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
result = str(index[pred[0]])
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
ModerateDemented
